In [150]:
# Initial imports
import pandas as pd
import numpy as np
from pathlib import Path
import hvplot.pandas
import matplotlib.pyplot as plt
import alpaca_trade_api as tradeapi
import os
import requests


In [151]:
# Load .env environment variables

import datetime as dt
import pytz

_timeNow = dt.datetime.now(pytz.timezone('US/Eastern'))
_1YearsAgo = _timeNow - dt.timedelta(days=360)
_2YearsAgo = _timeNow - dt.timedelta(days=730)
_3Years
Ago = _timeNow - dt.timedelta(days=1095)
ALPACA_API_BASE_URL = "https://paper-api.alpaca.markets/v2/assets?asset_class=crypto"
# Create a connection to the API 
api = tradeapi.REST("PKHBLCB7QI73L4912M0G", "Aq2JSZktyKcSKaFWq1VxIPn3EOFJB0ICvB7fdmpE", ALPACA_API_BASE_URL, api_version="v2")

In [188]:
# Set the ticker symbol and the number of shares to buy
ticker = "ETHUSD"
number_of_shares = 1

# Make API call
ETH_prices = api.get_crypto_bars(ticker, '1day',
                         start=_2YearsAgo.isoformat(),
                         end=None,
                         limit=None).df

In [189]:
#Filter data by exchange
ETH_prices = ETH_prices[ETH_prices.exchange == 'CBSE']

In [190]:
ETH_prices

,exchange,open,high,low,close,volume,trade_count,vwap
timestamp,,,,,,,,
2020-10-25 05:00:00+00:00,CBSE,406.32,411.47,403.19,408.25,15957.550316,7470,408.630560
2020-10-26 05:00:00+00:00,CBSE,393.07,396.31,392.01,394.73,13905.222140,6919,393.964598
2020-10-27 05:00:00+00:00,CBSE,405.75,409.52,404.43,405.89,16862.229051,7334,406.610429
2020-10-28 05:00:00+00:00,CBSE,388.59,392.29,386.76,390.73,22791.233054,12774,390.268853
2020-10-29 05:00:00+00:00,CBSE,390.73,394.00,380.75,387.60,81630.502392,41657,388.666188
...,...,...,...,...,...,...,...,...
2022-10-20 05:00:00+00:00,CBSE,1291.00,1311.10,1270.83,1288.05,275033.198898,389146,1291.612417
2022-10-21 05:00:00+00:00,CBSE,1288.07,1307.18,1254.00,1297.55,291607.681576,379933,1286.092632
2022-10-22 05:00:00+00:00,CBSE,1297.60,1320.56,1294.79,1309.93,101726.594934,232190,1310.274128


In [191]:
# Slice to just the `close` column
ETH_signals_df = ETH_prices.loc[:,["close"]]

In [192]:
# Set the short window and long windows
short_window = 50
long_window = 100

In [193]:
# Generate the short and long moving averages (50 and 100 days, respectively)
ETH_signals_df['SMA_fast'] = ETH_signals_df['close'].rolling(window=short_window).mean()
ETH_signals_df['SMA_slow'] = ETH_signals_df['close'].rolling(window=long_window).mean()

# Prepopulate the `Signal` for trading
ETH_signals_df['Signal'] = 0.0
ETH_signals_df = ETH_signals_df.dropna()
ETH_signals_df

,close,SMA_fast,SMA_slow,Signal
timestamp,,,,
2021-02-01 06:00:00+00:00,1389.48,1004.5124,750.9783,0.0
2021-02-02 06:00:00+00:00,1545.34,1023.7650,762.3492,0.0
2021-02-03 06:00:00+00:00,1657.11,1045.2234,774.9730,0.0
2021-02-04 06:00:00+00:00,1597.55,1064.2106,786.8896,0.0
2021-02-05 06:00:00+00:00,1662.98,1084.6028,799.6121,0.0
...,...,...,...,...
2022-10-20 05:00:00+00:00,1288.05,1415.9486,1521.0527,0.0
2022-10-21 05:00:00+00:00,1297.55,1410.0214,1522.9768,0.0
2022-10-22 05:00:00+00:00,1309.93,1404.9070,1524.1542,0.0


In [194]:
# Generate the trading signal 0 or 1,
# where 1 is when short-window (SMA50) is greater than the long (SMA 100)
# and 0 otherwise
ETH_signals_df['Signal'] = np.where(
    ETH_signals_df['SMA_fast']> ETH_signals_df['SMA_slow'], 1.0, 0.0)

In [195]:
ETH_signals_df

,close,SMA_fast,SMA_slow,Signal
timestamp,,,,
2021-02-01 06:00:00+00:00,1389.48,1004.5124,750.9783,1.0
2021-02-02 06:00:00+00:00,1545.34,1023.7650,762.3492,1.0
2021-02-03 06:00:00+00:00,1657.11,1045.2234,774.9730,1.0
2021-02-04 06:00:00+00:00,1597.55,1064.2106,786.8896,1.0
2021-02-05 06:00:00+00:00,1662.98,1084.6028,799.6121,1.0
...,...,...,...,...
2022-10-20 05:00:00+00:00,1288.05,1415.9486,1521.0527,0.0
2022-10-21 05:00:00+00:00,1297.55,1410.0214,1522.9768,0.0
2022-10-22 05:00:00+00:00,1309.93,1404.9070,1524.1542,0.0


In [196]:
# Use the pct_change function to generate returns from close prices
ETH_signals_df["Actual Returns"] = ETH_signals_df["close"].pct_change()

In [245]:
##### Calculate the points in time when the Signal value changes
# Identify trade entry (1) and exit (-1) points
ETH_signals_df['Entry/Exit'] = ETH_signals_df['Signal'].diff()

# Review the DataFrame
ETH_signals_df.loc["2021-02-02":"2022-06-20",:]

,close,SMA_fast,SMA_slow,Signal,Actual Returns,Entry/Exit,Position,Entry/Exit Position,Portfolio Holdings,Portfolio Cash,Portfolio Total
timestamp,,,,,,,,,,,
2021-02-02 06:00:00+00:00,1545.34,1023.7650,762.3492,1.0,0.112171,0.0,50.0,0.0,77267.0,100000.0,177267.0
2021-02-03 06:00:00+00:00,1657.11,1045.2234,774.9730,1.0,0.072327,0.0,50.0,0.0,82855.5,100000.0,182855.5
2021-02-04 06:00:00+00:00,1597.55,1064.2106,786.8896,1.0,-0.035942,0.0,50.0,0.0,79877.5,100000.0,179877.5
2021-02-05 06:00:00+00:00,1662.98,1084.6028,799.6121,1.0,0.040956,0.0,50.0,0.0,83149.0,100000.0,183149.0
2021-02-06 06:00:00+00:00,1616.61,1103.8704,811.9022,1.0,-0.027884,0.0,50.0,0.0,80830.5,100000.0,180830.5
...,...,...,...,...,...,...,...,...,...,...,...
2022-06-16 05:00:00+00:00,1084.99,2026.7366,2531.9766,0.0,-0.089147,0.0,0.0,0.0,0.0,148986.0,148986.0
2022-06-17 05:00:00+00:00,1079.22,1989.9510,2516.8402,0.0,-0.005318,0.0,0.0,0.0,0.0,148986.0,148986.0
2022-06-18 05:00:00+00:00,966.60,1952.8472,2500.8433,0.0,-0.104353,0.0,0.0,0.0,0.0,148986.0,148986.0


In [198]:
# Visualize exit position relative to close price
exit = ETH_signals_df[ETH_signals_df['Entry/Exit'] == -1.0]['close'].hvplot.scatter(
    color='yellow',
    marker='v',
    size=200,
    legend=False,
    ylabel='Price in $',
    width=1000,
    height=400
)

In [199]:
# Visualize entry position relative to close price
entry = ETH_signals_df[ETH_signals_df['Entry/Exit'] == 1.0]['close'].hvplot.scatter(
    color='purple',
    marker='^',
    size=200,
    legend=False,
    ylabel='Price in $',
    width=1000,
    height=400
)

In [200]:
# Visualize close price for the investment
security_close =ETH_signals_df[['close']].hvplot(
    line_color='lightgray',
    ylabel='Price in $',
    width=1000,
    height=400
)

In [201]:
# Visualize moving averages
moving_avgs = ETH_signals_df[['SMA_fast', 'SMA_slow']].hvplot(
    ylabel='Price in $',
    width=1000,
    height=400
)

In [202]:
moving_avgs

:NdOverlay   [Variable]
   :Curve   [timestamp]   (value)

In [203]:
# Create the overlay plot
entry_exit_plot = entry * exit

# Show the plot with a title
entry_exit_plot.opts(
    title="ETH - SMA_fast, SMA_slow, Entry and Exit Points"
)

:Overlay
   .Scatter.Close.I  :Scatter   [timestamp]   (close)
   .Scatter.Close.II :Scatter   [timestamp]   (close)

In [204]:
# Set initial capital
initial_capital = float(100000)

# Set the share size
share_size = 50

In [205]:
# Buy when the dual moving average crossover Signal equals 1
# Otherwise, `Position` should be zero (sell)
ETH_signals_df['Position'] = share_size * ETH_signals_df['Signal']

In [206]:
# Determine the points in time where a position is bought or sold
ETH_signals_df['Entry/Exit Position'] = ETH_signals_df['Position'].diff()

In [207]:
# Multiply the close price by the number of shares held, or the Position
ETH_signals_df['Portfolio Holdings'] = ETH_signals_df['close'] * ETH_signals_df['Position']

# Subtract the amount of either the cost or proceeds of the trade from the initial capital invested
ETH_signals_df['Portfolio Cash'] = initial_capital - (ETH_signals_df['close'] * ETH_signals_df['Entry/Exit Position']).cumsum() 

In [208]:
# Calculate the total portfolio value by adding the portfolio cash to the portfolio holdings (or investments)
ETH_signals_df['Portfolio Total'] = ETH_signals_df['Portfolio Cash'] + ETH_signals_df['Portfolio Holdings']

# Conclusion:  each point at a time, you can see what you have is 
# 1) if you are holding stock. say 500 share, what's holding value of 500 share at that moment + (initial capital + net of cost and proceed)
# in each row, you have  holding = position * close + initial investment - cost of previous buys and + proceed of previous sales

In [246]:
ETH_signals_df.loc["2021-02-02":"2022-10-20",:]

,close,SMA_fast,SMA_slow,Signal,Actual Returns,Entry/Exit,Position,Entry/Exit Position,Portfolio Holdings,Portfolio Cash,Portfolio Total
timestamp,,,,,,,,,,,
2021-02-02 06:00:00+00:00,1545.34,1023.7650,762.3492,1.0,0.112171,0.0,50.0,0.0,77267.0,100000.0,177267.0
2021-02-03 06:00:00+00:00,1657.11,1045.2234,774.9730,1.0,0.072327,0.0,50.0,0.0,82855.5,100000.0,182855.5
2021-02-04 06:00:00+00:00,1597.55,1064.2106,786.8896,1.0,-0.035942,0.0,50.0,0.0,79877.5,100000.0,179877.5
2021-02-05 06:00:00+00:00,1662.98,1084.6028,799.6121,1.0,0.040956,0.0,50.0,0.0,83149.0,100000.0,183149.0
2021-02-06 06:00:00+00:00,1616.61,1103.8704,811.9022,1.0,-0.027884,0.0,50.0,0.0,80830.5,100000.0,180830.5
...,...,...,...,...,...,...,...,...,...,...,...
2022-10-16 05:00:00+00:00,1305.25,1435.3886,1513.6507,0.0,0.015869,0.0,0.0,0.0,0.0,134820.0,134820.0
2022-10-17 05:00:00+00:00,1335.63,1433.1008,1515.0916,0.0,0.023275,0.0,0.0,0.0,0.0,134820.0,134820.0
2022-10-18 05:00:00+00:00,1303.57,1427.5268,1516.6421,0.0,-0.024004,0.0,0.0,0.0,0.0,134820.0,134820.0


In [210]:
#Train Test begin
from sklearn import svm
from sklearn.preprocessing import StandardScaler
from pandas.tseries.offsets import DateOffset
from sklearn.metrics import classification_report

In [211]:
# Assign a copy of the sma_fast and sma_slow columns to a features DataFrame called X
X = ETH_signals_df[['SMA_fast', 'SMA_slow']].shift().dropna()

# Review the DataFrame
X

,SMA_fast,SMA_slow
timestamp,,
2021-02-02 06:00:00+00:00,1004.5124,750.9783
2021-02-03 06:00:00+00:00,1023.7650,762.3492
2021-02-04 06:00:00+00:00,1045.2234,774.9730
2021-02-05 06:00:00+00:00,1064.2106,786.8896
2021-02-06 06:00:00+00:00,1084.6028,799.6121
...,...,...
2022-10-20 05:00:00+00:00,1421.2192,1518.6799
2022-10-21 05:00:00+00:00,1415.9486,1521.0527
2022-10-22 05:00:00+00:00,1410.0214,1522.9768


In [212]:
# Create the target set selecting the Signal column and assiging it to y
y = ETH_signals_df['Signal']

# Review the value counts
y.value_counts()

1.0    371
0.0    260
Name: Signal, dtype: int64

In [213]:
# Select the start of the training period
training_begin = X.index.min()

# Display the training begin date
print(training_begin)

2021-02-02 06:00:00+00:00


In [226]:
# Select the ending period for the training data with an offset of 1 months
training_end = X.index.min() + DateOffset(months=16)

# Display the training end date
print(training_end)

2022-06-02 06:00:00+00:00


In [227]:
# Generate the X_train and y_train DataFrames
X_train = X.loc[training_begin:training_end]
y_train = y.loc[training_begin:training_end]

# Review the X_train DataFrame
display(X_train.head())
display(X_train.tail())

,SMA_fast,SMA_slow
timestamp,,
2021-02-02 06:00:00+00:00,1004.5124,750.9783
2021-02-03 06:00:00+00:00,1023.7650,762.3492
2021-02-04 06:00:00+00:00,1045.2234,774.9730
2021-02-05 06:00:00+00:00,1064.2106,786.8896
2021-02-06 06:00:00+00:00,1084.6028,799.6121


,SMA_fast,SMA_slow
timestamp,,
2022-05-29 05:00:00+00:00,2553.9342,2731.7341
2022-05-30 05:00:00+00:00,2526.2970,2722.3468
2022-05-31 05:00:00+00:00,2502.3652,2715.5965
2022-06-01 05:00:00+00:00,2480.9964,2707.5302
2022-06-02 05:00:00+00:00,2456.0872,2700.6357


In [228]:
display(y_train.head())
display(y_train.tail())
y_train.describe()

timestamp
2021-02-02 06:00:00+00:00    1.0
2021-02-03 06:00:00+00:00    1.0
2021-02-04 06:00:00+00:00    1.0
2021-02-05 06:00:00+00:00    1.0
2021-02-06 06:00:00+00:00    1.0
Name: Signal, dtype: float64

timestamp
2022-05-29 05:00:00+00:00    0.0
2022-05-30 05:00:00+00:00    0.0
2022-05-31 05:00:00+00:00    0.0
2022-06-01 05:00:00+00:00    0.0
2022-06-02 05:00:00+00:00    0.0
Name: Signal, dtype: float64

count    486.000000
mean       0.666667
std        0.471890
min        0.000000
25%        0.000000
50%        1.000000
75%        1.000000
max        1.000000
Name: Signal, dtype: float64

In [229]:
X_train.describe()

,SMA_fast,SMA_slow
count,486.000000,486.000000
mean,2830.160751,2716.279031
std,809.346295,866.559217
min,1004.512400,750.978300
25%,2278.544000,2338.340900
50%,2856.430200,2745.826950
75%,3306.795700,3373.709650
max,4354.160600,4048.718200


In [230]:
# Generate the X_test and y_test DataFrames
X_test = X.loc[training_end+DateOffset(hours=1):]
y_test = y.loc[training_end+DateOffset(hours=1):]

# Review the X_test DataFrame
display(X_test.head())
display(X_test.tail())

,SMA_fast,SMA_slow
timestamp,,
2022-06-03 05:00:00+00:00,2430.1312,2692.1991
2022-06-04 05:00:00+00:00,2404.8098,2686.5368
2022-06-05 05:00:00+00:00,2379.6808,2678.3559
2022-06-06 05:00:00+00:00,2356.4846,2669.3639
2022-06-07 05:00:00+00:00,2333.2638,2659.7898


,SMA_fast,SMA_slow
timestamp,,
2022-10-20 05:00:00+00:00,1421.2192,1518.6799
2022-10-21 05:00:00+00:00,1415.9486,1521.0527
2022-10-22 05:00:00+00:00,1410.0214,1522.9768
2022-10-23 05:00:00+00:00,1404.9070,1524.1542
2022-10-24 05:00:00+00:00,1400.7362,1525.4126


In [231]:
# Scale the features DataFrames

# Create a StandardScaler instance
scaler = StandardScaler()

# Apply the scaler model to fit the X-train data
X_scaler = scaler.fit(X_train)

# Transform the X_train and X_test DataFrames using the X_scaler
X_train_scaled = X_scaler.transform(X_train)
X_test_scaled = X_scaler.transform(X_test)

In [232]:
# From SVM, instantiate SVC classifier model instance
svm_model = svm.SVC()

# Fit the model to the data using the training data
svm_model = svm_model.fit(X_train_scaled, y_train)
 
# Use the testing data to make the model predictions
svm_pred = svm_model.predict(X_test_scaled)

# Review the model's predicted values
svm_pred[:10]
        

array([0., 0., 0., 0., 0., 0., 0., 0., 0., 0.])

In [233]:
# Use a classification report to evaluate the model using the predictions and testing data
svm_testing_report = classification_report(y_test, svm_pred)

# Print the classification report
print(svm_testing_report)

              precision    recall  f1-score   support

         0.0       1.00      0.41      0.58        98
         1.0       0.44      1.00      0.61        46

    accuracy                           0.60       144
   macro avg       0.72      0.70      0.60       144
weighted avg       0.82      0.60      0.59       144



In [234]:
# Create a new empty predictions DataFrame:

# Create a predictions DataFrame
predictions_df = pd.DataFrame(index=X_test.index)

# Add the SVM model predictions to the DataFrame
predictions_df['Predicted'] = svm_pred

# Add the actual returns to the DataFrame
predictions_df['Actual Returns'] = ETH_signals_df["Actual Returns"]

# Add the strategy returns to the DataFrame
predictions_df['Strategy Returns'] = ETH_signals_df["Actual Returns"]*predictions_df["Predicted"]

# Review the DataFrame
display(predictions_df.head())
display(predictions_df.tail())

,Predicted,Actual Returns,Strategy Returns
timestamp,,,
2022-06-03 05:00:00+00:00,0.0,-0.026565,-0.0
2022-06-04 05:00:00+00:00,0.0,0.012762,0.0
2022-06-05 05:00:00+00:00,0.0,0.049657,0.0
2022-06-06 05:00:00+00:00,0.0,-0.067011,-0.0
2022-06-07 05:00:00+00:00,0.0,0.019698,0.0


,Predicted,Actual Returns,Strategy Returns
timestamp,,,
2022-10-20 05:00:00+00:00,1.0,-0.002270,-0.002270
2022-10-21 05:00:00+00:00,1.0,0.007375,0.007375
2022-10-22 05:00:00+00:00,1.0,0.009541,0.009541
2022-10-23 05:00:00+00:00,1.0,0.028986,0.028986
2022-10-24 05:00:00+00:00,1.0,-0.000015,-0.000015


In [238]:
# Plot the actual returns versus the strategy returns
(1 + predictions_df[["Actual Returns", "Strategy Returns"]]).cumprod().hvplot(width=1000,height=500)

:NdOverlay   [Variable]
   :Curve   [timestamp]   (value)